# loop

在编译原理课程项目中，进行自然循环识别的代码。

## 需要的库

- 正则表达式 re
- 序列化 json

In [1]:
import re
import json
import copy

## 读入json

In [52]:
filename = "../examples/quicksort_src_blk.json"
vm_blk = {}
with open(filename, "r") as fp:
    vm_blk = json.load(fp)
blocks = vm_blk["blocks"]

cond_f = {"==": lambda x, y: x == y, "!=": lambda x, y: x != y, ">=": lambda x, y: x >=
          y, "<=": lambda x, y: x <= y, ">": lambda x, y: x > y, "<": lambda x, y: x < y}
arith_f = {"+": lambda x, y: x+y, "-": lambda x, y: x-y, "*": lambda x,
           y: x*y, "/": lambda x, y: x//y, "%": lambda x, y: x % y}
eserved_word = {"HALT", "=", "+", "-", "*", "/", "%", "?",
                ":", "!:", ">", "<", ">=", "<=", "==", "!=", "[", "]"}
reg = re.compile('^[0-9]+$')

In [53]:
blocks

{'0': {'line_num': [0, 3],
  'next': [1, None],
  'code': ['SL [ 0 ] = 0', 'N = N - 1', 'SR [ 0 ] = N', 'TOP = 0'],
  'defd': ['SL', 'SR', 'TOP'],
  'used': ['N'],
  'in': ['A', 'N'],
  'out': ['SR', 'SL', 'A', 'TOP']},
 '1': {'line_num': [4, 4],
  'next': [2, 9],
  'code': ['? TOP < 0 : 47'],
  'defd': [],
  'used': ['TOP'],
  'in': ['SL', 'A', 'SR', 'TOP'],
  'out': ['A', 'SL', 'SR', 'TOP']},
 '2': {'line_num': [5, 8],
  'next': [3, 1],
  'code': ['M = SL [ TOP ]',
   'N = SR [ TOP ]',
   'TOP = TOP - 1',
   '? N <= M : 4'],
  'defd': ['N', 'M'],
  'used': ['SL', 'SR', 'TOP'],
  'in': ['SL', 'A', 'SR', 'TOP'],
  'out': ['M', 'SR', 'N', 'SL', 'A', 'TOP']},
 '3': {'line_num': [9, 12],
  'next': [4, None],
  'code': ['I = M - 1', 'J = N', 'T_1 = N', 'V = A [ T_1 ]'],
  'defd': ['J', 'V', 'I', 'T_1'],
  'used': ['A', 'N', 'M'],
  'in': ['N', 'A', 'M', 'TOP'],
  'out': ['M', 'J', 'I', 'V', 'N', 'A', 'TOP']},
 '4': {'line_num': [13, 16],
  'next': [5, 4],
  'code': ['I = I + 1', 'T_2 = I',

# DFS并分配行号

In [54]:
stack = ['0']
visited = ['0']

line_count = 0
while len(stack)>0:
    tmp = stack[-1]
    stack.pop()
    blocks[tmp]["entry_line_num"] = line_count
    print(tmp,":",line_count)
    if "!:" in blocks[tmp]["code"][-1]:
        line_count += len(blocks[tmp]["code"])
    elif blocks[tmp]["code"][-1] == "HALT":
        line_count += len(blocks[tmp]["code"])
    else:
        line_count += len(blocks[tmp]["code"]) + 1
    
    for i in reversed(blocks[tmp]["next"]) :
        if (i!=None):
            if str(i) not in visited:
                stack.append(str(i))
                visited.append(str(i))

out_code = ['' for i in range(line_count)]

0 : 0
1 : 5
2 : 7
3 : 12
4 : 17
5 : 22
6 : 27
7 : 29
8 : 38
9 : 54


In [55]:
for n,b in blocks.items():
    if len(blocks[n]["code"])==1:
        blocks[n]["code"] = list(blocks[n]["code"])
    if b["next"][1]!=None:
        symbols = blocks[n]["code"][-1].split(" ")
        symbols[-1] = str(blocks[str(b["next"][1])]["entry_line_num"])
        blocks[n]["code"][-1] = " ".join(symbols)
    if b["next"][0]!=None:
        if "!:" in blocks[n]["code"][-1]:
            blocks[n]["code"][-1] = "!: {}".format(blocks[str(b["next"][0])]["entry_line_num"])
        else:
            blocks[n]["code"].append("!: {}".format(blocks[str(b["next"][0])]["entry_line_num"]))
    for i,l in enumerate(blocks[n]["code"]):
        out_code[blocks[n]["entry_line_num"]+i] = l
    print(blocks[n]["code"])

['SL [ 0 ] = 0', 'N = N - 1', 'SR [ 0 ] = N', 'TOP = 0', '!: 5']
['? TOP < 0 : 54', '!: 7']
['M = SL [ TOP ]', 'N = SR [ TOP ]', 'TOP = TOP - 1', '? N <= M : 5', '!: 12']
['I = M - 1', 'J = N', 'T_1 = N', 'V = A [ T_1 ]', '!: 17']
['I = I + 1', 'T_2 = I', 'T_3 = A [ T_2 ]', '? V > T_3 : 17', '!: 22']
['J = J - 1', 'T_4 = J', 'T_5 = A [ T_4 ]', '? T_5 > V : 22', '!: 27']
['? I >= J : 38', '!: 29']
['T_6 = I', 'X = A [ T_6 ]', 'T_7 = I', 'T_8 = J', 'T_9 = A [ T_8 ]', 'A [ T_7 ] = T_9', 'T_10 = J', 'A [ T_10 ] = X', '!: 17']
['T_11 = I', 'X = A [ T_11 ]', 'T_12 = I', 'T_13 = N', 'T_14 = A [ T_13 ]', 'A [ T_12 ] = T_14', 'T_15 = N', 'A [ T_15 ] = X', 'TOP = TOP + 1', 'SL [ TOP ] = M', 'SR [ TOP ] = J', 'TOP = TOP + 1', 'T_16 = I + 1', 'SL [ TOP ] = T_16', 'SR [ TOP ] = N', '!: 5']
['HALT']


In [58]:
for i,l in enumerate(out_code):
    print("{:4d} -   {}".format(i,l))

   0 -   SL [ 0 ] = 0
   1 -   N = N - 1
   2 -   SR [ 0 ] = N
   3 -   TOP = 0
   4 -   !: 5
   5 -   ? TOP < 0 : 54
   6 -   !: 7
   7 -   M = SL [ TOP ]
   8 -   N = SR [ TOP ]
   9 -   TOP = TOP - 1
  10 -   ? N <= M : 5
  11 -   !: 12
  12 -   I = M - 1
  13 -   J = N
  14 -   T_1 = N
  15 -   V = A [ T_1 ]
  16 -   !: 17
  17 -   I = I + 1
  18 -   T_2 = I
  19 -   T_3 = A [ T_2 ]
  20 -   ? V > T_3 : 17
  21 -   !: 22
  22 -   J = J - 1
  23 -   T_4 = J
  24 -   T_5 = A [ T_4 ]
  25 -   ? T_5 > V : 22
  26 -   !: 27
  27 -   ? I >= J : 38
  28 -   !: 29
  29 -   T_6 = I
  30 -   X = A [ T_6 ]
  31 -   T_7 = I
  32 -   T_8 = J
  33 -   T_9 = A [ T_8 ]
  34 -   A [ T_7 ] = T_9
  35 -   T_10 = J
  36 -   A [ T_10 ] = X
  37 -   !: 17
  38 -   T_11 = I
  39 -   X = A [ T_11 ]
  40 -   T_12 = I
  41 -   T_13 = N
  42 -   T_14 = A [ T_13 ]
  43 -   A [ T_12 ] = T_14
  44 -   T_15 = N
  45 -   A [ T_15 ] = X
  46 -   TOP = TOP + 1
  47 -   SL [ TOP ] = M
  48 -   SR [ TOP ] = J
  49 - 

# 删除无用goto

In [74]:
opt_goto = {}
for i,l in enumerate(out_code):
    symbols = l.split(" ")
    if (symbols[0]=="!:") and (int(symbols[-1])==i+1):
        print("{:4d} -   {}  (DEL)".format(i,l))
    else:
        print("{:4d} -   {}".format(i,l))
        opt_goto[i]=symbols

   0 -   SL [ 0 ] = 0
   1 -   N = N - 1
   2 -   SR [ 0 ] = N
   3 -   TOP = 0
   4 -   !: 5  (DEL)
   5 -   ? TOP < 0 : 54
   6 -   !: 7  (DEL)
   7 -   M = SL [ TOP ]
   8 -   N = SR [ TOP ]
   9 -   TOP = TOP - 1
  10 -   ? N <= M : 5
  11 -   !: 12  (DEL)
  12 -   I = M - 1
  13 -   J = N
  14 -   T_1 = N
  15 -   V = A [ T_1 ]
  16 -   !: 17  (DEL)
  17 -   I = I + 1
  18 -   T_2 = I
  19 -   T_3 = A [ T_2 ]
  20 -   ? V > T_3 : 17
  21 -   !: 22  (DEL)
  22 -   J = J - 1
  23 -   T_4 = J
  24 -   T_5 = A [ T_4 ]
  25 -   ? T_5 > V : 22
  26 -   !: 27  (DEL)
  27 -   ? I >= J : 38
  28 -   !: 29  (DEL)
  29 -   T_6 = I
  30 -   X = A [ T_6 ]
  31 -   T_7 = I
  32 -   T_8 = J
  33 -   T_9 = A [ T_8 ]
  34 -   A [ T_7 ] = T_9
  35 -   T_10 = J
  36 -   A [ T_10 ] = X
  37 -   !: 17
  38 -   T_11 = I
  39 -   X = A [ T_11 ]
  40 -   T_12 = I
  41 -   T_13 = N
  42 -   T_14 = A [ T_13 ]
  43 -   A [ T_12 ] = T_14
  44 -   T_15 = N
  45 -   A [ T_15 ] = X
  46 -   TOP = TOP + 1
  47 -

In [75]:
opt_goto_list = sorted(list(opt_goto.items()))
line_num_map = {opt_goto_list[i][0]:i for i in range(len(opt_goto_list))}
for li in opt_goto_list:
    l = li[1]
    if "!:"==l[0] or "?" == l[0]:
        l[-1] = str(line_num_map[int(l[-1])])
    print("{:4d} -   {}".format(line_num_map[li[0]]," ".join(l)))

   0 -   SL [ 0 ] = 0
   1 -   N = N - 1
   2 -   SR [ 0 ] = N
   3 -   TOP = 0
   4 -   ? TOP < 0 : 47
   5 -   M = SL [ TOP ]
   6 -   N = SR [ TOP ]
   7 -   TOP = TOP - 1
   8 -   ? N <= M : 4
   9 -   I = M - 1
  10 -   J = N
  11 -   T_1 = N
  12 -   V = A [ T_1 ]
  13 -   I = I + 1
  14 -   T_2 = I
  15 -   T_3 = A [ T_2 ]
  16 -   ? V > T_3 : 13
  17 -   J = J - 1
  18 -   T_4 = J
  19 -   T_5 = A [ T_4 ]
  20 -   ? T_5 > V : 17
  21 -   ? I >= J : 31
  22 -   T_6 = I
  23 -   X = A [ T_6 ]
  24 -   T_7 = I
  25 -   T_8 = J
  26 -   T_9 = A [ T_8 ]
  27 -   A [ T_7 ] = T_9
  28 -   T_10 = J
  29 -   A [ T_10 ] = X
  30 -   !: 13
  31 -   T_11 = I
  32 -   X = A [ T_11 ]
  33 -   T_12 = I
  34 -   T_13 = N
  35 -   T_14 = A [ T_13 ]
  36 -   A [ T_12 ] = T_14
  37 -   T_15 = N
  38 -   A [ T_15 ] = X
  39 -   TOP = TOP + 1
  40 -   SL [ TOP ] = M
  41 -   SR [ TOP ] = J
  42 -   TOP = TOP + 1
  43 -   T_16 = I + 1
  44 -   SL [ TOP ] = T_16
  45 -   SR [ TOP ] = N
  46 -   !: 4
